### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
import random

from sklearn.naive_bayes import ComplementNB

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# Solución:

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

## Parte 1

In [ ]:
def get_random_sample(doc_list, doc_len):

  doc_list_final = []
  doc_list_target = []
  doc_list_target_names = []

  rand_list = random.sample(range(1, len(doc_list.data)), doc_len)

  for i in rand_list:

    doc_list_final.append(doc_list.data[i])
    doc_list_target.append(doc_list.target[i])
    doc_list_target_names.append(doc_list.target_names[doc_list.target[i]])

  return doc_list_final, doc_list_target, doc_list_target_names

In [ ]:
# tomando tamaño de muestra aleatorio:

sample_size = 100

rand_documents, rand_targets, doc_list_target_names = get_random_sample(newsgroups_train, sample_size)

rand_documents[0:1]

['HEY!!! All you Yankee fans who\'ve been knocking my prediction of Baltimore.\nYou flooded my mailbox with cries of "Militello\'s good, Militello\'s good."\n\nWhere is he??!! I noticed he got skipped over after that oh so strong first\nouting.  He\'s not by any chance in Columbus  now, is he?  Please don\'t tell\nme you\'re relying on this guy to be the *fourth*, not the fifth, but the \n*fourth* starter on this brittle pitching staff.  \n\nAs for the O\'s, it\'s still early.\n\n\nSee y\'all at the ballyard\nGo Braves\nChop Chop\n\nMichael Mule\'\n']

In [ ]:
X_train = tfidfvect.fit_transform(rand_documents)

y_train = rand_targets
#y_train[:10]
y_train[0:5]

[9, 11, 8, 16, 10]

In [ ]:
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (100, 3492)
cantidad de documentos: 100
tamaño del vocabulario (dimensionalidad de los vectores): 3492


In [ ]:
# Tomemos 5 documentos aleatorios

# midamos la similaridad coseno de cada documento aleatorio con todos los documentos de train
cossim = []

for idx in range(5):

  cossim.append(cosine_similarity(X_train[idx], X_train)[0])

np.argsort(cossim[4])[::-1][1:6]

array([44, 15, 33, 53, 41])

In [ ]:
# los 5 documentos más similares:
mostsim = []

for idx in range(5):

  mostsim.append(np.argsort(cossim[idx])[::-1][1:6])

mostsim

[array([33, 72, 97, 79, 44]),
 array([33, 41, 72, 98, 90]),
 array([47, 88, 12, 41, 76]),
 array([33, 90, 41,  7, 72]),
 array([44, 15, 33, 53, 41])]

In [ ]:
# y los 5 más similares son de las clases:
for idx in range(5):
  print('-------------------------------------------------')
  print(f'{idx} doc type {doc_list_target_names[idx]}:')
  print('-------------------------------------------------')
  print(f'Doc num {idx}:')
  for i in mostsim[idx]:
    print('*****')
    print(f'{idx} similar to doc {i}??:')
    print(doc_list_target_names[i])
    print('*****')
    print(rand_documents[i][0:10])

-------------------------------------------------
0 doc type rec.sport.baseball:
-------------------------------------------------
Doc num 0:
*****
0 similar to doc 33??:
talk.politics.misc
*****
In respond
*****
0 similar to doc 72??:
talk.religion.misc
*****


There we
*****
0 similar to doc 97??:
rec.motorcycles
*****
Hi

I t wa
*****
0 similar to doc 79??:
talk.politics.mideast
*****



Is ther
*****
0 similar to doc 44??:
rec.sport.baseball
*****
No, he's n
-------------------------------------------------
1 doc type sci.crypt:
-------------------------------------------------
Doc num 1:
*****
1 similar to doc 33??:
talk.politics.misc
*****
In respond
*****
1 similar to doc 41??:
sci.space
*****
Archive-na
*****
1 similar to doc 72??:
talk.religion.misc
*****


There we
*****
1 similar to doc 98??:
talk.politics.guns
*****


But what
*****
1 similar to doc 90??:
rec.autos
*****


  I won'
-------------------------------------------------
2 doc type rec.motorcycles:
---------------

In [ ]:
# Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

## Parte 2:

In [ ]:
# Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test.

# Vectorizar documentos

tfidfvect = TfidfVectorizer(
                            stop_words='english', # 0.646
                            smooth_idf=False # 0.648
                            )

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6482017249372933

In [ ]:
# Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB

from sklearn.naive_bayes import ComplementNB

clf = ComplementNB()

clf.fit(X_train, y_train)

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_score(y_test, y_pred, average='macro')

0.6937086080234727

## Parte 3:

In [ ]:
X_train_T = X_train.T

In [ ]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:

def get_term_similarity(idx,X_T):

  print(f"Termino: {idx}" )
  print(idx2word[idx])
  print("*****")
  cossim = cosine_similarity(X_T[idx], X_T)[0]

  np.sort(cossim)[::-1]
  np.argsort(cossim)[::-1]

  mostsim = np.argsort(cossim)[::-1][1:6]

  for i in mostsim:
    print(idx2word[i])

  print("-----------------------------------")

In [ ]:
# Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [ ]:
indices_terminos = [26001, 30001, 35001, 41001, 46001]

for i in indices_terminos:
    get_term_similarity(i,X_train_T)

Termino: 26001
categorical


categorical
_just_
elephants
_no_
omniscient
-----------------------------------
Termino: 30001
cottage


therpy
aske
noew
hopeully
beacause
-----------------------------------
Termino: 35001
drawer


deceased
belonged
sterile
modification
3671
-----------------------------------
Termino: 41001
fools


agrevates
brotherman
bullsh
abord
rids
-----------------------------------
Termino: 46001
heath


x10
arround
programing
dimmer
reliably
-----------------------------------
